In [15]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
import os,time,datetime

### Structure of the PINNs
first for burgers equation

full batch

2 -> 7x20 -> 1

In [3]:
in_dim = 2
out_dim = 1
width = 20
depth = 7

epoch = 8000
tol = 1e-8

N_0 = 50
N_b = 50
N_r = 2000

w_init = "Glorot"
b_init = "zeros"
act = "tanh"

lr = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate = 5e-3,
    decay_steps = epoch,
    alpha = 1e-2
)

opt = "Adam"
info_freq = 100
info_seed = 1234

weight_data = 1.
weight_pde = 1.

print("python    :", sys.version)
print("tensorflow:", tf.__version__)
print("rand seed :", info_seed)
os.environ["PYTHONHASHSEED"] = str(info_seed)
np.random.seed(info_seed)
tf.random.set_seed(info_seed)

python    : 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]
tensorflow: 2.9.1
rand seed : 1234


### Data for Burgers equation

$$
\begin{aligned}
&u_t+uu_x−(0.01/\pi)u_{xx}=0,   x\in[−1,1],   t\in[0,1] \\
&u(0,x)=−sin(\pi x)\\
&u(t,−1)=u(t,1)=0
\end{aligned}
$$

In [14]:
tmin, tmax =  0., 1.
xmin, xmax = -1., 1.
lb = tf.constant([tmin, xmin], dtype = tf.float64)
ub = tf.constant([tmax, xmax], dtype = tf.float64)

t_0 = tf.ones((N_0, 1), dtype = tf.float64) * lb[0]
x_0 = tf.random.uniform((N_0, 1), lb[1], ub[1], dtype = tf.float64)
t_b = tf.random.uniform((N_b, 1), lb[0], ub[0], dtype = tf.float64)
x_b = lb[1] + (ub[1] - lb[1]) * tf.keras.backend.random_bernoulli((N_b, 1), .5, dtype = tf.float64)
t_r = tf.random.uniform((N_r, 1), lb[0], ub[0], dtype = tf.float64)
x_r = tf.random.uniform((N_r, 1), lb[1], ub[1], dtype = tf.float64)

# initial and boundary
u_0 = -tf.sin(np.pi*x_0)
u_b = tf.zeros((x_b.shape[0],1),dtype=tf.float64)

### Build PINNs

In [ ]:
class PINN(tf.keras.Model):
    def __init__(
            self,
            x,t,u,x_r,u_r,lb,up
            in_dim,out_dim,width,depth,
            activ="tanh",w_init="glorot_normal",b_init="zeros",
            lr=1e-3,opt="Adam",weight_data=1.,weight_pde=1.,
            info_freq=100,info_seed=1234):
        # information
        self.info_freq = info_freq
        self.info_seed = info_freq
        # initial the data
        self.data_type = tf.float64
        self.x = tf.convert_to_tensor(x,dtype=self.data_type)
        self.t = tf.convert_to_tensor(t,dtype=self.data_type)
        self.u = tf.convert_to_tensor(u,dtype=self.data_type)
        self.x_r = tf.convert_to_tensor(x_r,dtype=self.data_type)
        self.u_r = tf.convert_to_tensor(u_r,dtype=self.data_type)
        self.lb = tf.convert_to_tensor(lb,dtype=self.data_type)
        self.ub = tf.convert_to_tensor(ub,dtype=self.data_type)
        self.nu = tf.constant(0.01/np.pi,dtype=self.data_type)
        # neuron network configuration
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.width = width
        self.depth = depth
        self.activ = activ
        self.w_init = w_init
        self.b_init = b_init
        self.lr = lr
        self.opt = opt
        self.weight_data = weight_data
        self.weight_pde = weight_pde
        
        # call
        self.dnn = self.dnn_init(in_dim,out_dim,width,depth)
        self.params = self.dnn.trainable_variables
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = lr, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)

        # track loss
        self.ep_log = []
        self.loss_log = []

        print("\n************************************************************")
        print("****************     MAIN PROGRAM START     ****************")
        print("************************************************************")
        print(">>>>> start time:", datetime.datetime.now())
        print(">>>>> configuration;")
        print("         dtype        :", self.data_type)
        print("         activ func   :", self.activ)
        print("         weight init  :", self.w_init)
        print("         learning rate:", self.lr)
        print("         optimizer    :", self.opt)
        print("         summary      :", self.dnn.summary())
    
    def dnn_init(in_dim,out_dim,width,depth):
        net = tf.keras.Sequential()
        net.add(tf.keras.layers.InputLayer(in_dim))
        net.add(tf.keras.layers.Lambda(lambda x: 2. * (x - self.lb) / (self.ub - self.lb) - 1.))

        for l in range(depth - 1):
            net.add(tf.keras.layers.Dense(units=width, activation = activ,kernel_initializer = w_init, bias_initializer = b_init, ))
        net.add(tf.keras.layers.Dense(out_dim))
        return net
    
    def loss_pde(self):
        # with tf.GradientTape(persistent=True) as tp
        with tf.GradientTape() as tp:
            with tf.GradientTape() as tp2:
                tp.watch(self.x_r)
                tp.watch(self.t_r)
                u = self.dnn(tf.concat([self.t_r,self.x_r],1))
            u_t = tp2.gradient(u,t)
            u_x = tp2.gradient(u,x)
        u_xx = tp.gradient(u_x,x)
        del tp,tp2
        gv = u_t + u * u_x - self.nu * u_xx
        r = tf.reduce_mean(tf.square(gv))
        return r

    def loss_icbc(self):
        



In [20]:
x = tf.convert_to_tensor([1.,2.])
# A = tf.convert_to_tensor([1.,2.])
with tf.GradientTape(persistent=True) as tp:
    tp.watch(x)
    z = x**3
del tp
print(x)
print(z)
dz = tp.gradient(z,x)
ddz = tp.jacobian(z,x)
print(dz)
print(ddz)


tf.Tensor([1. 2.], shape=(2,), dtype=float32)
tf.Tensor([1. 8.], shape=(2,), dtype=float32)


NameError: name 'tp' is not defined